In [1]:
import requests
import math

api_carpark_availability = 'https://api.data.gov.sg/v1/transport/carpark-availability'  # This dataset doesn't contain the coods of carpark
api_pollutant_standard = 'https://api.data.gov.sg/v1/environment/psi'
api_air_temperature = 'https://api.data.gov.sg/v1/environment/air-temperature'
api_ultra_violet = 'https://api.data.gov.sg/v1/environment/uv-index'
api_pm25 = 'https://api.data.gov.sg/v1/environment/pm25'
apiList = [api_carpark_availability, api_pollutant_standard, api_air_temperature, api_ultra_violet, api_pm25]

def _check_api_availability():
  ava = True
  for api in apiList:
    response = requests.get(api)
    if response.status_code != 200:
      ava = False
  return ava

def _find_closest_cood(la, lo, dic):
  # dic: {'name of cood':[la, lo]}
  # return the name of the cood which is closest to the input (la,lo)
  closest_key = ''
  min_distance = 999
  for key in dic:
    cood = dic[key]
    distance = math.sqrt((la-cood[0])**2 + (lo-cood[1])**2)
    # print(la,lo,cood)
    # print(key, distance)
    if distance < min_distance:
      closest_key = key
      min_distance = distance
  return closest_key

def _get_pollutant_standard(la,lo):
  response = requests.get(api_pollutant_standard)

  temp = response.json()

  region_metadata = temp['region_metadata']
  coods = {}
  for region in region_metadata:
    name = region['name']
    region_la = region['label_location']['latitude']
    region_lo = region['label_location']['longitude']
    coods[name] = [region_la, region_lo]
  # print(coods)
  
  best_key = _find_closest_cood(la, lo, coods)
  # print(best_key)

  update_timestamp = temp['items'][0]['update_timestamp']
  # print(update_timestamp)

  readings = temp['items'][0]['readings']
  # print(readings)
  values = {}
  for key in readings:
    values[key] = readings[key][best_key]

  return values, update_timestamp

# _get_pollutant_standard(1.35735,103.94)

def _get_carpark():
  response = requests.get(api_carpark_availability)
  temp = response.json()['items'][0]
  print(type(temp))
  print(temp.keys())

# _get_carpark()

def _get_pm25_by_coordinate(la,lo):
  # print(la, lo)
  response = requests.get(api_pm25)

  update_timestamp = (response.json()['items'])[0]['update_timestamp']
  values = (response.json()['items'])[0]['readings']['pm25_one_hourly']
  
  locs = (response.json()['region_metadata'])
  coods = {}
  for loc in locs:
    name = loc['name']
    loc_la = loc['label_location']['latitude']
    loc_lo = loc['label_location']['longitude']
    coods[name] = [loc_la,loc_lo]

  # print(values)
  # print(coods)
  best_key = _find_closest_cood(la, lo, coods)
  return values[best_key], update_timestamp

# _get_pm25_by_coordinate(1.41803,103.82)

def _get_uv():
  response = requests.get(api_ultra_violet)
  temp = response.json()['items'][0]['index']
  # print(temp)
  latestData = temp[0]
  return latestData['value'], latestData['timestamp']

# _get_uv()

def _get_air_temp_by_coordinate(la,lo):
  response = requests.get(api_air_temperature)

  temp = response.json()

  metadata = temp['metadata']['stations']
  coods = {}
  for station in metadata:
    station_id = station['id']
    station_la = station['location']['latitude']
    station_lo = station['location']['longitude']
    coods[station_id] = [station_la, station_lo]
  # print(coods)

  items = temp['items'][0]
  timestamp = items['timestamp']
  readings = items['readings']
  values = {}
  for station in readings:
    id = station['station_id']
    value = station['value']
    values[id] = value
  # print(values)

  best_key = _find_closest_cood(la, lo, coods)
  # print(best_key)

  return values[best_key], timestamp

# _get_air_temp_by_coordinate(1.3135,103.9625)

In [2]:
_get_air_temp_by_coordinate(1.3135,103.9625)

(27.1, '2022-04-10T02:30:00+08:00')

In [3]:
_get_uv()

(0, '2022-04-09T19:00:00+08:00')

In [5]:
_get_pm25_by_coordinate(1.3135,103.9625)

{'west': 7, 'east': 18, 'central': 19, 'south': 11, 'north': 22}


(18, '2022-04-10T02:08:52+08:00')

In [7]:
_get_pollutant_standard(1.3135,103.9625)

({'o3_sub_index': 12,
  'pm10_twenty_four_hourly': 36,
  'pm10_sub_index': 36,
  'co_sub_index': 3,
  'pm25_twenty_four_hourly': 13,
  'so2_sub_index': 7,
  'co_eight_hour_max': 0.29,
  'no2_one_hour_max': 24,
  'so2_twenty_four_hourly': 11,
  'pm25_sub_index': 52,
  'psi_twenty_four_hourly': 52,
  'o3_eight_hour_max': 29},
 '2022-04-10T00:08:54+08:00')

In [8]:
temp, time = _get_air_temp_by_coordinate(1.3135,103.9625)

In [9]:
time

'2022-04-10T01:45:00+08:00'

In [17]:
time.split('T')[1].split(':')

['01', '45', '00+08', '00']

In [ ]:
h